# Finetuning модели на новых данных

В этом задании вам предстоит обучить предобученную модель для классификации изображений на новых данных.

## Настройка google colab

Для корректной работы ноутбука и отправки заданий в тестирующую систему запустите следующие две ячейки (до начала работы).

In [ ]:
%%bash

rm colab_setup

wget -q https://raw.githubusercontent.com/hse-cs-ami/coursera-intro-dl/main/utils/colab_setup.py -O colab_setup.py

In [ ]:
import colab_setup

colab_setup.Week03CNN2().setup()

In [ ]:
from testing import TestWeek03


tester = TestWeek03()

tester.set_email('### YOUR EMAIL ###')
tester.set_token('### YOUR TOKEN ###')

## Необходимые импорты

In [ ]:
import os
from collections import defaultdict
from glob import glob
from time import perf_counter
from warnings import filterwarnings

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.transforms as t

from IPython.display import clear_output
from PIL import Image
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
from tqdm import tqdm

filterwarnings('ignore')

sns.set(style='darkgrid')

## Задание 1. Датасет

Заполните пропуски в коде, чтобы получился рабочий класс для датасета.

In [ ]:
class Birbs(Dataset):

    def __init__(self, base_path: str = './birbs', train: bool = True, transform = None):
        self.base_path = # объедините base_path и 'train' если train иначе 'test'

        self.filenames = sorted(glob(os.path.join(self.base_path, '*/*')))
        self.labels = # сделайте тензор с типом torch.LongTensor, используйте функцию get_label

        self.transform = transform

    def get_label(self, filename):
        return int(filename.split('/')[-2]) - 1

    def __getitem__(self, idx: int):
        filename = # достаньте filename из filenames

        image = Image.open(filename)

        if self.transform is not None:
            image = # примените transform для преобразования image

        return image, self.labels[idx]

    def __len__(self):
        return len(self.filenames)

In [ ]:
# тестируем как вы написали класс датасета
tester.set_email('### YOUR EMAIL ###')
tester.set_token('### YOUR TOKEN ###')

tester.test01(Birbs)

## Задание 2. Загрузка модели и подготовка к обучению на новых данных

Заполните пропуски в коде, чтобы получился рабочий класс модели.

Запустите ячейку ниже и посмотрите на устройство модели [ResNet](https://arxiv.org/abs/1512.03385).

In [ ]:
resnet = resnet18(pretrained=True)

print(resnet)

В своей реализации мы заменим последний полносвязный слой (`fc`) на новый полносвязный слой (который будет соответствовать нашей задаче - иметь нужное количество классов 200).

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = resnet18(pretrained=True)

        num_in_features = # получите количество входных признаков слоя fc модели resnet (используйте in_features)
        num_classes = 200

        self.model.fc = # создайте слой Linear с вышезаданными параметрами

    def forward(self, x):
        return # сделайте проход вперед с помощью model

    @property
    def input_size(self):
        return 224

model = Model()

In [ ]:
# тестируем как вы написали класс модели
tester.set_email('### YOUR EMAIL ###')
tester.set_token('### YOUR TOKEN ###')

tester.test02(model)

## Задание 3. Обучение на выбранном датасете

Заполните пропуски в коде, чтобы получился рабочий класс оболочки.

In [ ]:
class Wrapper:

    def __init__(self, model: Model, train_transform: t.Compose = None):

        # задаём аугментации для обучающей и тестовой выборок
        if train_transform is None:
            train_transform: t.Compose = t.Compose(
                [
                    t.Resize(model.input_size),
                    t.ToTensor(),
                    t.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]
            )

        test_transform: t.Compose = t.Compose(
            [
                t.Resize(model.input_size),
                t.ToTensor(),
                t.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]
        )

        # создаем датасеты
        train_dataset = # создайте обучающий датасет
        test_dataset = # создайте тестовый датасет

        batch_size = 32

        # создаем даталоадеры
        self.train_loader: DataLoader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
        self.test_loader: DataLoader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

        self.device: torch.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

        self.model: Model = model.to(self.device)

        # создаем функцию потерь и оптимизатор
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = Adam(self.model.parameters(), lr=1e-3)

        self.history = defaultdict(list)

    def run(self, epochs: int = 10):
        total_time = 0

        for epoch in range(epochs):
            # строим графики
            self.plot_stats(epochs)

            # обучаемся
            start = perf_counter()
            self.train_epoch()
            total_time += perf_counter() - start

            # оцениваем качество
            self.evaluate_subset('train')
            self.evaluate_subset('test')

        # строим финальные графики и печатаем сколько заняло обучение
        self.plot_stats(epochs)
        print(f'Время на обучение: {total_time:.2f} секунд')

    def train_epoch(self):
        self.model.train()

        for images, labels in tqdm(self.train_loader, desc='Обучение'):
            images, labels = images.to(self.device), labels.to(self.device)

            # обнулите градиент у оптимизатора

            # получите выход модели на батче images

            # посчитайте ошибку между выходом модели и labels, запишите в loss

            loss.backward()

            self.optimizer.step()

    @torch.inference_mode()
    def evaluate_subset(self, subset: str = 'train'):
        self.model.eval()

        loss = 0.0
        correct = 0

        loader: DataLoader = self.train_loader if subset == 'train' else self.test_loader

        for images, labels in tqdm(loader, desc=f'Оценка качества на {subset}'):
            images, labels = images.to(self.device), labels.to(self.device)

            # получите выход модели на батче images, запишите в outputs

            # посчитайте ошибку между выходом модели и labels, запишите в loss, используйте .item() для получения числа

            _, prediction = torch.max(outputs, 1)
            correct += prediction.eq(labels.data.view_as(prediction)).cpu().sum()

        self.history[f'{subset}_loss'].append(loss / len(loader))
        self.history[f'{subset}_accuracy'].append(100. * correct / len(loader.dataset))

    def plot_stats(self, epochs: int):
        clear_output(wait=True)

        fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 10), constrained_layout=True)

        ax1.set_title('Зависимость ошибки модели от номера эпохи')
        ax1.plot(
            range(1, len(self.history['train_loss']) + 1),
            self.history['train_loss'],
            label='Обучающая', marker='^'
        )
        ax1.plot(
            range(1, len(self.history['test_loss']) + 1),
            self.history['test_loss'],
            label='Тестовая', marker='^'
        )

        ax1.set_xlim([0.5, epochs + 0.5])

        ax1.set_xlabel('Эпоха')
        ax1.set_ylabel('Ошибка')

        ax1.legend()

        ax2.set_title('Зависимость доли правильных ответов модели от номера эпохи')
        ax2.plot(
            range(1, len(self.history['train_accuracy']) + 1),
            self.history['train_accuracy'],
            label='Обучающая', marker='^'
        )
        ax2.plot(
            range(1, len(self.history['test_accuracy']) + 1),
            self.history['test_accuracy'],
            label='Тестовая', marker='^'
        )

        ax2.set_xlim([0.5, epochs + 0.5])

        ax2.set_xlabel('Эпоха')
        ax2.set_ylabel('Доля правильных ответов')

        ax2.legend()

        plt.show()

Попробуйте использовать какие-нибудь аугментации из списка:

 - [RandomHorizontalFlip](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomHorizontalFlip)
 - [RandomAffine](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomAffine)
 - [ColorJitter](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ColorJitter)
 - [RandomGrayscale](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomGrayscale)
 - [RandomCrop](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomCrop)

Или используйте любую другую аугментацию по вашему вкусу.

In [ ]:
custom_transform = t.Compose(
    [
        # добавьте сюда аугментации из списка выше
        t.Resize(model.input_size),
        t.ToTensor(),
        t.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
# создайте оболочку
wrapper = Wrapper(model=model, train_transform=custom_transform)

Ваша цель добиться качества NN% на тестовой выборке.

In [ ]:
# запустите обучение
wrapper.run()

In [ ]:
# запустите тестирование обученной модели
tester.set_email('### YOUR EMAIL ###')
tester.set_token('### YOUR TOKEN ###')

tester.test03(wrapper)